# Numpy Array Operations:  Axes, Broadcasting, Matrix Type

In this note we cover a few concepts about `numpy` multi-dimensional arrays in more detail:
* Using the `axis` feature 
* Python broadcasting

We will need both of these for performing many of the numerical operations for the ML class.

As usual, we begin by loading the `numpy` package.

In [1]:
import numpy as np

## Axis Parameter

Many operations in the `numpy` package can take an optional `axis` parameter to specify which dimensions the operation is to be applied.  This is extremely useful for multi-dimensional data.  To illustrate the `axis` parameter, consider a matrix the `(3,2)` array `X` defined as:

In [2]:
X = np.array([[0,1],[2,3],[4,5]])
print(X)

[[0 1]
 [2 3]
 [4 5]]


An operation like `np.mean` or `np.sum` takes the mean or sum of *all* elements in the array -- from all rows and columns. 

In [3]:
print(np.mean(X))
print(np.sum(X))

2.5
15


To take only the `sum` along each column, we can use the `axis` parameter.

In [4]:
print(np.sum(X,axis=0))

[6 9]


Since `X` has shape `(3,2)`, the output `np.sum(X,axis=0)` is of shape `(2,)`.  Similarly, we can take the `sum` along each row:

In [5]:
print(np.sum(X,axis=1))

[1 5 9]


You can apply this to higher-order arrays. Here we create a 3D array to see.

In [6]:
X = np.arange(24).reshape(2,3,4)  # shape = (2,3,4)
print(X)

[[[ 0  1  2  3]
  [ 4  5  6  7]
  [ 8  9 10 11]]

 [[12 13 14 15]
  [16 17 18 19]
  [20 21 22 23]]]


In [7]:
Y1 = np.sum(X,axis=0)             # shape = (3,4)
Y2 = np.sum(X,axis=1)             # shape = (2,4)
print('Y1 = ')
print(Y1)
print('Y2 = ')
print(Y2)

Y1 = 
[[12 14 16 18]
 [20 22 24 26]
 [28 30 32 34]]
Y2 = 
[[12 15 18 21]
 [48 51 54 57]]


## Broadcasting

**Broadcasting** is a useful tool in Python for performing operations on matrices. It generalizes the useful fact that, if we multiply a numpy array by a scalar, Python knows that we want to multiply *every entry* by that scalar.  

In [8]:
a = np.array([[1,2,3],[4,5,6]])

b = 2
print(b*a)
print(a*b)

[[ 2  4  6]
 [ 8 10 12]]
[[ 2  4  6]
 [ 8 10 12]]


### Example 1:  Mean Removal

Suppose that `X` is a data matrix of shape `(n,p)`.  That is, there are `n` data points and `p` features per point.  Often, we have to remove the mean from each feature.  That is, we want to compute the mean for each feature and then remove the mean from each column.  We could do this with a for-loop as:

In [9]:
# Generate some random data
n = 100
p = 5
X = np.random.rand(n,p)

In [10]:
Xm = np.zeros(p)      # Mean for each feature
X_demean = np.zeros((n,p))  # Transformed features with the means removed
for j in range(p):
    Xm[j] = np.mean(X[:,j])
    for i in range(n):
        X_demean[i,j] = X[i,j] - Xm[j]
print(X_demean[0:7,:]) # print the first several rows of the result to compare to later

[[ 0.32699171 -0.05827885  0.12387888  0.20546108 -0.47732498]
 [-0.01890085 -0.00484674 -0.26613385  0.10590703 -0.18742857]
 [ 0.12578217  0.0084754   0.35474042 -0.39153716 -0.21711182]
 [-0.05106496 -0.16018461 -0.19992087 -0.45990573  0.14258305]
 [-0.11905086 -0.26533337 -0.14087846  0.47132141  0.41344269]
 [-0.1258379  -0.31642671  0.05105157  0.49670571 -0.22709146]
 [-0.13319531  0.43780288  0.06890912 -0.43838166 -0.08214898]]


The code below does this without a for loop using the `axis` parameter and broadcasting.

In [11]:
# Compute the mean per column using the axis command
Xm = np.mean(X,axis=0)  # This is a p-dim matrix
print(Xm)

[0.45014744 0.51643577 0.50786066 0.46955971 0.49626589]


To use broadcasting we will need to convert Xm to a 2 dimensional ndarray. We can do this with the `Xm[None,:]` operation, which returns a `(1,p)` shape array.

In [12]:
print(Xm[None,:])

[[0.45014744 0.51643577 0.50786066 0.46955971 0.49626589]]


Using Python broadcasting, we can then subtract the `Xm[None,:]` from `X`. These array are different sizes -- `Xm[None,:]` has one row while `X` has n. But Python automatically figures out that, since the number of columns match, we want to substract `Xm[None,:]` off of every row in X.

In [13]:
# Subtract the mean
X_demean = X - Xm[None,:]
print(X_demean[0:7,:])

[[ 0.32699171 -0.05827885  0.12387888  0.20546108 -0.47732498]
 [-0.01890085 -0.00484674 -0.26613385  0.10590703 -0.18742857]
 [ 0.12578217  0.0084754   0.35474042 -0.39153716 -0.21711182]
 [-0.05106496 -0.16018461 -0.19992087 -0.45990573  0.14258305]
 [-0.11905086 -0.26533337 -0.14087846  0.47132141  0.41344269]
 [-0.1258379  -0.31642671  0.05105157  0.49670571 -0.22709146]
 [-0.13319531  0.43780288  0.06890912 -0.43838166 -0.08214898]]


### Example 2:  Standardizing variables

A variant of the above example is to *standardize* the features, where we compute the transform variables,

    Z[i,j] = (X[i,j] - Xm[j])/ Xstd[j]
    
where `Xstd[j]` is the standard deviation per feature.  This can be done as follows:

In [14]:
Xstd = np.std(X,axis=0)
Z = (X-Xm[None,:])/Xstd[None,:]


### Example 3:  Outer product

The *outer product* of vectors `x` and `y` is the matrix `Z[i,j] = x[i]y[j]`.  This can be performed in one line as follows

In [40]:
# Some random data
nx = 100
ny = 10
x = np.array([1, 2, 3])
y = np.array([5, 6, 7])

print(x.shape)
print(y.shape)

# Compute the outer product in one line
Z = x[:,None]*y[None,:]
print(x[:,None].shape)
print(y[None,:].shape)
print(Z)


(3,)
(3,)
(3, 1)
(1, 3)
[[ 5  6  7]
 [10 12 14]
 [15 18 21]]


Here:

     x[:,None] # Has shape (nx,  1)
     y[None,:] # Has shape ( 1, ny)
     
So, with python broadcasting:

     Z = x[:,None]*y[None,:] # has shape (nx,  ny)


**Exercise 1:**  Given a matrix `X`, compute the matrix `Y`, where the rows of `X` are normaized to have norm one.  That is:

     Y[i,j] = X[i,j] / sum_j X[i,j]   

In [16]:
X = np.random.rand(4,3)
# Y = ...


**Exercise 2:** Diagonal multiplication.  Given a matrix `X` and a vector `d`, compute `Y = diag(d)*X`.

In [44]:
X = np.random.rand(5,3)
d = np.random.rand(5)

print(np.diag(d).shape)
print(X.shape)

Y = np.diag(d) @ X

(5, 5)
(5, 3)


## Matrix operations with numpy 
Python broadcasting is great, but sometimes it does exactly the wrong thing! If you have a column vector `z` and a matrix `X`, `X*z` won't compute the matrix vector product, but rather use broadcasting to scales `X`'s rows. Similarly for matrix `X` and `Y`, `X*Y` does not compute an actual matrix product.

In [18]:
X = np.array([[1,2],[3,4]])
z = np.array([[2],[3]])
Y = np.array([[-1,-1],[-1,-1]])
print(str(X)+'\n\n'+str(z)+'\n\n'+str(Y))

[[1 2]
 [3 4]]

[[2]
 [3]]

[[-1 -1]
 [-1 -1]]


In [19]:
print(str(X*z));print()
print(X*Y)

[[ 2  4]
 [ 9 12]]

[[-1 -2]
 [-3 -4]]


To compute actually matrix/matrix and matrix/vector products, you can use the `dot` operation.

In [20]:
print(np.dot(X,z));print()
print(np.dot(np.transpose(z),z));print()
print(np.dot(X,Y));print()

[[ 8]
 [18]]

[[13]]

[[-3 -3]
 [-7 -7]]



Alternatively, a much cleaner approach is to use numpy's `@` operator. This operator works directly on 2D ndarrays with compatible dimensions. **You do not need to convert to a matrix type as we did in the other demo**. Apparently that approach is outdated, and the `@` operator is now preferred. 

In [21]:
print(X@z);print()
print(np.transpose(z)@z);print()
print(X@Y)

[[ 8]
 [18]]

[[13]]

[[-3 -3]
 [-7 -7]]


Note that `@` between a row and column vector gives an alternative way to compute outerproducts.

In [22]:
print(z@np.transpose(z))

[[4 6]
 [6 9]]
